# Week 05 — Probability & Noise (Likelihood)

This notebook ties loss-based ML to probabilistic modeling through maximum likelihood estimation. You'll:
- Understand how likelihood assumptions lead to common loss functions
- Implement MLE for different noise models
- Explore robust alternatives to Gaussian assumptions

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize
import seaborn as sns
%matplotlib inline

np.random.seed(42)
sns.set_style('whitegrid')
print("Libraries imported!")

## 1. Maximum Likelihood Estimation (MLE) for Gaussian Linear Regression

Derive and implement MLE, showing the connection between negative log-likelihood and mean squared error.

In [ ]:
# Generate synthetic data
n_samples = 100
X = np.random.randn(n_samples, 1) * 2
w_true = np.array([2.0, 3.5])  # [intercept, slope]
sigma_true = 1.0

# Add bias column
X_with_bias = np.hstack([np.ones((n_samples, 1)), X])
y_true = X_with_bias @ w_true
y = y_true + np.random.randn(n_samples) * sigma_true

# Define negative log-likelihood for Gaussian noise
def neg_log_likelihood_gaussian(params, X, y):
    """
    Negative log-likelihood for linear regression with Gaussian noise
    
    L(w, sigma | X, y) = -sum(log N(y_i | X_i @ w, sigma^2))
                       = (n/2) log(2*pi*sigma^2) + sum((y_i - X_i @ w)^2) / (2*sigma^2)
    """
    n = len(y)
    w = params[:-1]
    sigma = params[-1]
    
    if sigma <= 0:
        return 1e10  # Invalid sigma
    
    mu = X @ w
    residuals = y - mu
    
    # Negative log-likelihood
    nll = 0.5 * n * np.log(2 * np.pi * sigma**2) + np.sum(residuals**2) / (2 * sigma**2)
    return nll

# Optimize to find MLE
initial_params = np.array([0.0, 0.0, 1.0])  # [w0, w1, sigma]
result = minimize(neg_log_likelihood_gaussian, initial_params, args=(X_with_bias, y), method='L-BFGS-B')

w_mle = result.x[:-1]
sigma_mle = result.x[-1]

print("True parameters: w0={:.2f}, w1={:.2f}, sigma={:.2f}".format(*w_true, sigma_true))
print("MLE parameters:  w0={:.2f}, w1={:.2f}, sigma={:.2f}".format(*w_mle, sigma_mle))

# Plot results
plt.figure(figsize=(10, 5))
plt.scatter(X, y, alpha=0.6, label='Data')
X_line = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
X_line_bias = np.hstack([np.ones((100, 1)), X_line])
y_pred_mle = X_line_bias @ w_mle
y_pred_true = X_line_bias @ w_true

plt.plot(X_line, y_pred_true, 'g--', linewidth=2, label='True model')
plt.plot(X_line, y_pred_mle, 'r-', linewidth=2, label='MLE fit')
plt.fill_between(X_line.ravel(), 
                 y_pred_mle - 2*sigma_mle, 
                 y_pred_mle + 2*sigma_mle, 
                 alpha=0.2, color='red', label='±2σ')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Maximum Likelihood Estimation')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## 2. Connection: NLL and MSE

Show that minimizing negative log-likelihood with Gaussian noise is equivalent to minimizing MSE.

In [ ]:
# For Gaussian noise with fixed sigma, NLL ∝ MSE
# Let's verify this by computing both

def mse_loss(w, X, y):
    """Mean Squared Error"""
    mu = X @ w
    return np.mean((y - mu)**2)

# Grid search over w1 (fixing w0 and sigma)
w0_fixed = w_mle[0]
sigma_fixed = sigma_mle
w1_range = np.linspace(0, 6, 100)

nll_values = []
mse_values = []

for w1 in w1_range:
    params = np.array([w0_fixed, w1, sigma_fixed])
    nll = neg_log_likelihood_gaussian(params, X_with_bias, y)
    nll_values.append(nll)
    
    mse = mse_loss(np.array([w0_fixed, w1]), X_with_bias, y)
    mse_values.append(mse)

# Normalize for comparison
nll_normalized = (nll_values - np.min(nll_values)) / (np.max(nll_values) - np.min(nll_values))
mse_normalized = (mse_values - np.min(mse_values)) / (np.max(mse_values) - np.min(mse_values))

plt.figure(figsize=(10, 5))
plt.plot(w1_range, nll_normalized, 'b-', linewidth=2, label='Normalized NLL')
plt.plot(w1_range, mse_normalized, 'r--', linewidth=2, label='Normalized MSE')
plt.axvline(w_true[1], color='g', linestyle=':', linewidth=2, label='True w1')
plt.axvline(w_mle[1], color='orange', linestyle=':', linewidth=2, label='MLE w1')
plt.xlabel('w1 (slope parameter)')
plt.ylabel('Normalized Loss')
plt.title('Negative Log-Likelihood vs MSE\n(Curves overlap when noise is Gaussian!)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print("Correlation between NLL and MSE:", np.corrcoef(nll_values, mse_values)[0, 1])

## 3. Robust Regression: Laplace and Student-t Noise

Explore robust alternatives to Gaussian noise that are less sensitive to outliers.

In [ ]:
# Generate data with outliers
np.random.seed(42)
X_robust = np.random.randn(80, 1) * 2
X_robust_bias = np.hstack([np.ones((80, 1)), X_robust])
y_robust = X_robust_bias @ w_true + np.random.randn(80) * 0.5

# Add outliers
outlier_idx = [10, 25, 40, 55, 70]
y_robust[outlier_idx] += np.random.choice([-5, 5], size=len(outlier_idx))

# MLE with Laplace noise (L1 loss)
def neg_log_likelihood_laplace(params, X, y):
    """NLL for Laplace noise: L1 loss"""
    w = params[:-1]
    b = params[-1]  # scale parameter
    if b <= 0:
        return 1e10
    
    mu = X @ w
    residuals = np.abs(y - mu)
    return len(y) * np.log(2 * b) + np.sum(residuals) / b

# MLE with Student-t noise (robust to outliers)
def neg_log_likelihood_t(params, X, y, df=3):
    """NLL for Student-t noise"""
    w = params[:-1]
    scale = params[-1]
    if scale <= 0:
        return 1e10
    
    mu = X @ w
    nll = -np.sum(stats.t.logpdf((y - mu) / scale, df=df)) + len(y) * np.log(scale)
    return nll

# Fit all three models
init = np.array([0.0, 0.0, 1.0])

result_gaussian = minimize(neg_log_likelihood_gaussian, init, args=(X_robust_bias, y_robust), method='L-BFGS-B')
result_laplace = minimize(neg_log_likelihood_laplace, init, args=(X_robust_bias, y_robust), method='L-BFGS-B')
result_t = minimize(neg_log_likelihood_t, init, args=(X_robust_bias, y_robust), method='L-BFGS-B')

# Plot comparison
plt.figure(figsize=(12, 5))
plt.scatter(X_robust, y_robust, alpha=0.6, s=50, label='Data')
plt.scatter(X_robust[outlier_idx], y_robust[outlier_idx], color='red', s=100, 
            marker='x', linewidths=3, label='Outliers')

X_line = np.linspace(X_robust.min(), X_robust.max(), 100).reshape(-1, 1)
X_line_bias = np.hstack([np.ones((100, 1)), X_line])

plt.plot(X_line, X_line_bias @ w_true, 'g--', linewidth=2, label='True model', alpha=0.7)
plt.plot(X_line, X_line_bias @ result_gaussian.x[:-1], 'b-', linewidth=2, label='Gaussian MLE')
plt.plot(X_line, X_line_bias @ result_laplace.x[:-1], 'orange', linewidth=2, label='Laplace MLE (L1)')
plt.plot(X_line, X_line_bias @ result_t.x[:-1], 'purple', linewidth=2, label='Student-t MLE')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Robust Regression: Gaussian vs Laplace vs Student-t Noise Models')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print("\nModel Comparison:")
print(f"Gaussian: w0={result_gaussian.x[0]:.2f}, w1={result_gaussian.x[1]:.2f}")
print(f"Laplace:  w0={result_laplace.x[0]:.2f}, w1={result_laplace.x[1]:.2f}")
print(f"Student-t: w0={result_t.x[0]:.2f}, w1={result_t.x[1]:.2f}")
print(f"\nTrue:     w0={w_true[0]:.2f}, w1={w_true[1]:.2f}")
print("\n→ Laplace and Student-t are more robust to outliers!")

## 4. Likelihood Surfaces and Multimodality

Visualize likelihood as a function of parameters to understand optimization landscapes.

In [ ]:
# Create a grid over w0 and w1
w0_range = np.linspace(0, 4, 50)
w1_range = np.linspace(1, 6, 50)
W0, W1 = np.meshgrid(w0_range, w1_range)

# Compute NLL for each combination
NLL = np.zeros_like(W0)
sigma_fixed = 1.0

for i in range(len(w0_range)):
    for j in range(len(w1_range)):
        params = np.array([W0[j, i], W1[j, i], sigma_fixed])
        NLL[j, i] = neg_log_likelihood_gaussian(params, X_with_bias, y)

# Plot likelihood surface
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
contour = plt.contour(W0, W1, NLL, levels=30, cmap='viridis')
plt.colorbar(contour, label='Negative Log-Likelihood')
plt.plot(w_mle[0], w_mle[1], 'r*', markersize=20, label='MLE')
plt.plot(w_true[0], w_true[1], 'g*', markersize=20, label='True')
plt.xlabel('w0 (intercept)')
plt.ylabel('w1 (slope)')
plt.title('Likelihood Surface (Contours)')
plt.legend()
plt.grid(alpha=0.3)

# 3D surface
from mpl_toolkits.mplot3d import Axes3D
ax = plt.subplot(1, 2, 2, projection='3d')
surf = ax.plot_surface(W0, W1, NLL, cmap='viridis', alpha=0.6, edgecolor='none')
ax.scatter([w_mle[0]], [w_mle[1]], [neg_log_likelihood_gaussian(np.concatenate([w_mle, [sigma_fixed]]), X_with_bias, y)],
           color='red', s=100, label='MLE')
ax.set_xlabel('w0')
ax.set_ylabel('w1')
ax.set_zlabel('NLL')
ax.set_title('Likelihood Surface (3D)')
plt.tight_layout()
plt.show()

print("The likelihood surface is convex for Gaussian linear regression.")
print("This guarantees a unique global maximum (minimum NLL).")

## Exercises for Further Practice

1. **Poisson Regression**: Implement MLE for count data using Poisson likelihood
2. **Bayesian Linear Regression**: Add priors and compute MAP estimates
3. **Heteroscedastic Noise**: Model variance as a function of inputs
4. **AIC/BIC**: Implement model selection using information criteria
5. **Real Data**: Apply different noise models to UCI regression datasets

## Deliverables Checklist

- [ ] MLE derivation and implementation for Gaussian linear regression
- [ ] Demonstration of NLL ≈ MSE connection
- [ ] Robust regression comparison (Gaussian vs Laplace vs Student-t)
- [ ] Likelihood surface visualization with interpretation

## Recommended Resources

- Murphy, "Machine Learning: A Probabilistic Perspective" (MLE chapter)
- Bishop, "Pattern Recognition and Machine Learning" (Probability chapter)
- SciPy documentation on probability distributions